In [2]:
import os
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from copy import copy
from typing import Union, Optional

import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.formatting.rule import Rule
from openpyxl.styles.differential import DifferentialStyle

from datetime import date, datetime

In [244]:
def copy_excel_cell_range(
        src_ws: openpyxl.worksheet.worksheet.Worksheet,
        min_row: int = None,
        max_row: int = None,
        min_col: int = None,
        max_col: int = None,
        tgt_ws: openpyxl.worksheet.worksheet.Worksheet = None,
        tgt_min_row: int = 1,
        tgt_min_col: int = 1,
        with_style: bool = True
) -> openpyxl.worksheet.worksheet.Worksheet:
    """
    copies all cells from the source worksheet [src_ws] starting from [min_row] row
    and [min_col] column up to [max_row] row and [max_col] column
    to target worksheet [tgt_ws] starting from [tgt_min_row] row
    and [tgt_min_col] column.

    @param src_ws:  source worksheet
    @param min_row: smallest row index in the source worksheet (1-based index)
    @param max_row: largest row index in the source worksheet (1-based index)
    @param min_col: smallest column index in the source worksheet (1-based index)
    @param max_col: largest column index in the source worksheet (1-based index)
    @param tgt_ws:  target worksheet.
                    If None, then the copy will be done to the same (source) worksheet.
    @param tgt_min_row: target row index (1-based index)
    @param tgt_min_col: target column index (1-based index)
    @param with_style:  whether to copy cell style. Default: True

    @return: target worksheet object
    """
    if tgt_ws is None:
        tgt_ws = src_ws

    # https://stackoverflow.com/a/34838233/5741205
    for row in src_ws.iter_rows(min_row=min_row, max_row=max_row,
                                min_col=min_col, max_col=max_col):
        for cell in row:
            tgt_cell = tgt_ws.cell(
                row=cell.row + tgt_min_row - 1,
                column=cell.col_idx + tgt_min_col - 1,
                value=cell.value
            )
            if with_style and cell.has_style:
                # tgt_cell._style = copy(cell._style)
                tgt_cell.font = copy(cell.font)
                tgt_cell.border = copy(cell.border)
                tgt_cell.fill = copy(cell.fill)
                tgt_cell.number_format = copy(cell.number_format)
                tgt_cell.protection = copy(cell.protection)
                tgt_cell.alignment = copy(cell.alignment)
    return tgt_ws


def append_df_to_excel(
        filename: Union[str, Path],
        df: pd.DataFrame,
        sheet_name: str = 'Sheet1',
        startrow: Optional[int] = None,
        max_col_width: int = 30,
        autofilter: bool = False,
        fmt_int: str = "#,##0",
        fmt_float: str = "#,##0.00",
        fmt_date: str = "yyyy-mm-dd",
        fmt_datetime: str = "yyyy-mm-dd hh:mm",
        truncate_sheet: bool = False,
        storage_options: Optional[dict] = None,
        **to_excel_kwargs
) -> None:
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    @param filename: File path or existing ExcelWriter
                     (Example: '/path/to/file.xlsx')
    @param df: DataFrame to save to workbook
    @param sheet_name: Name of sheet which will contain DataFrame.
                       (default: 'Sheet1')
    @param startrow: upper left cell row to dump data frame.
                     Per default (startrow=None) calculate the last row
                     in the existing DF and write to the next row...
    @param max_col_width: maximum column width in Excel. Default: 40
    @param autofilter: boolean - whether add Excel autofilter or not. Default: False
    @param fmt_int: Excel format for integer numbers
    @param fmt_float: Excel format for float numbers
    @param fmt_date: Excel format for dates
    @param fmt_datetime: Excel format for datetime's
    @param truncate_sheet: truncate (remove and recreate) [sheet_name]
                           before writing DataFrame to Excel file
    @param storage_options: dict, optional
        Extra options that make sense for a particular storage connection, e.g. host, port,
        username, password, etc., if using a URL that will be parsed by fsspec, e.g.,
        starting “s3://”, “gcs://”.
    @param to_excel_kwargs: arguments which will be passed to `DataFrame.to_excel()`
                            [can be a dictionary]
    @return: None

    Usage examples:

    >>> append_df_to_excel('/tmp/test.xlsx', df, autofilter=True,
                           freeze_panes=(1,0))

    >>> append_df_to_excel('/tmp/test.xlsx', df, header=None, index=False)

    >>> append_df_to_excel('/tmp/test.xlsx', df, sheet_name='Sheet2',
                           index=False)

    >>> append_df_to_excel('/tmp/test.xlsx', df, sheet_name='Sheet2',
                           index=False, startrow=25)

    >>> append_df_to_excel('/tmp/test.xlsx', df, index=False,
                           fmt_datetime="dd.mm.yyyy hh:mm")

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    def set_column_format(ws, column_letter, fmt):
        for cell in ws[column_letter]:
            cell.number_format = fmt
    filename = Path(filename)
    file_exists = filename.is_file()
    # process parameters
    # calculate first column number
    # if the DF will be written using `index=True`, then `first_col = 2`, else `first_col = 1`
    first_col = int(to_excel_kwargs.get("index", True)) + 1
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
    # save content of existing sheets
    if file_exists:
        wb = load_workbook(filename)
        sheet_names = wb.sheetnames
        sheet_exists = sheet_name in sheet_names
        sheets = {ws.title: ws for ws in wb.worksheets}

    with pd.ExcelWriter(
        filename.with_suffix(".xlsx"),
        engine="openpyxl",
        mode="a" if file_exists else "w",
        if_sheet_exists="new" if file_exists else None,
        date_format=fmt_date,
        datetime_format=fmt_datetime,
        storage_options=storage_options
    ) as writer:
        if file_exists:
            # try to open an existing workbook
            writer.book = wb
            # get the last row in the existing Excel sheet
            # if it was not specified explicitly
            if startrow is None and sheet_name in writer.book.sheetnames:
                startrow = writer.book[sheet_name].max_row
            # truncate sheet
            if truncate_sheet and sheet_name in writer.book.sheetnames:
                # index of [sheet_name] sheet
                idx = writer.book.sheetnames.index(sheet_name)
                # remove [sheet_name]
                writer.book.remove(writer.book.worksheets[idx])
                # create an empty sheet [sheet_name] using old index
                writer.book.create_sheet(sheet_name, idx)
            # copy existing sheets
            writer.sheets = sheets
        else:
            # file doesn't exist, we are creating a new one
            startrow = 0

        # write out the DataFrame to an ExcelWriter
        df.to_excel(writer, sheet_name=sheet_name, **to_excel_kwargs)
        worksheet = writer.sheets[sheet_name]

        if autofilter:
            worksheet.auto_filter.ref = worksheet.dimensions

        for xl_col_no, dtyp in enumerate(df.dtypes, first_col):
            col_no = xl_col_no - first_col
            width = max(df.iloc[:, col_no].astype(str).str.len().max(),
                        len(df.columns[col_no]) + 6)
            width = min(max_col_width, width)
            column_letter = get_column_letter(xl_col_no)
            worksheet.column_dimensions[column_letter].width = width
            if np.issubdtype(dtyp, np.integer):
                set_column_format(worksheet, column_letter, fmt_int)
            if np.issubdtype(dtyp, np.floating):
                set_column_format(worksheet, column_letter, fmt_float)

    if file_exists and sheet_exists:
        # move (append) rows from new worksheet to the `sheet_name` worksheet
        wb = load_workbook(filename)
        # retrieve generated worksheet name
        new_sheet_name = set(wb.sheetnames) - set(sheet_names)
        if new_sheet_name:
            new_sheet_name = list(new_sheet_name)[0]
        # copy rows written by `df.to_excel(...)` to
        copy_excel_cell_range(
            src_ws=wb[new_sheet_name],
            tgt_ws=wb[sheet_name],
            tgt_min_row=startrow + 1,
            with_style=True
        )
        # remove new (generated by Pandas) worksheet
        del wb[new_sheet_name]
        wb.save(filename)
        wb.close()

In [245]:
# CONSTANTS
FILE_EXTENSION = ".txt"
FILEPATH = "./files"
TABLES_SEPARATER = '\n\n'
TABLE_DATA_SEPRATER = '\n'
TABLENAME_STARTING_DELIMITER = "*** "
TABLENAME_ENDING_DELIMTER = " ***"
COLUMNS_SEPARATER = '\t'
TABLE_DATA_SEPARATER = '\t'

# Filename Convention Implementation_check_<current date>_<current time>.xlsx
RESULT_FILENAME = "Implementation_check_" + str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")) + ".xlsx"
SHEET_NAME_EXTRACTOR = ""

# Replaces any spaces, &, - characters into underscore
TABLENAME_CLEANING_CHARACTERS = [(" ", "_"),("&", "_"),("-", "_")]

In [246]:
requiredFilesList = list()

with os.scandir(FILEPATH) as currentDirectory:
    for entry in currentDirectory:
        
        # Criteria for selecting the required files
        criteriaSet = set()
        criteriaSet.add(entry.name.endswith(FILE_EXTENSION))
        criteriaSet.add((entry.name.lower().find("before") > -1) or (entry.name.lower().find("after") > -1))
        if False not in criteriaSet:
            requiredFilesList.append(entry.name)

# Updating the file names to include the path
requiredFilesList = [FILEPATH + "/" + file for file in requiredFilesList]
requiredFilesList.sort()
requiredFilesList

['./files/test2After.txt', './files/test2Before.txt']

In [247]:
resultList = dict()
dxf2 = DifferentialStyle(fill = PatternFill(bgColor = "FFC7CE"))# Red Fill
dxf = DifferentialStyle(fill = PatternFill(bgColor = "00CCFFCC"))# Green Fill
g = Rule(type = "expression", dxf = dxf2, stopIfTrue = True)
r = Rule(type = "expression", dxf = dxf, stopIfTrue = True)

for entry in requiredFilesList:

    fileType = "after"
    if entry[:-len(FILE_EXTENSION)][-1] == "e": fileType = "before"
    
    comparisonList = list()
    dbName = entry[ : -len(FILE_EXTENSION) - len(fileType)]
    con = sqlite3.connect(dbName + '.db')
    cur = con.cursor()
    file=open(entry,"r")
    
    for table in file.read().split(TABLES_SEPARATER):
        
        # Get the data of individual tables
        tableInfo = table.split(TABLE_DATA_SEPRATER)
        
        if tableInfo != '' and len(tableInfo) > 1:
            
            # Cleaning and storing the table name
            tableName = tableInfo[0][len(TABLENAME_STARTING_DELIMITER) : -len(TABLENAME_ENDING_DELIMTER)]
            tableName = tableName + "_" + fileType
            for clean_char in TABLENAME_CLEANING_CHARACTERS:
                tableName = tableName.replace(clean_char[0], clean_char[1])
            
            # Cleaning and storing the table columns
            tableColumns = list()
            for column in tableInfo[1].split(COLUMNS_SEPARATER):
                tableColumns.append(column.replace(".", "_"))
            tableColumns = tuple(tableColumns)
            
            # Creating tables and execution
            tableCreationQuery = 'CREATE TABLE {table_name} {table_columns}'
            tableCreationQuery = tableCreationQuery.format(table_name = tableName, table_columns = tableColumns)
            try:
                cur.execute(tableCreationQuery)
            except sqlite3.OperationalError:
                pass
            
            for data in tableInfo[2:]:
                
                # Creating table data and execution
                row_data = tuple(data.split(TABLE_DATA_SEPARATER))
                tableDataInsertionQuery = 'INSERT INTO {table_name} VALUES {table_data}'
                tableDataInsertionQuery = tableDataInsertionQuery.format(table_name = tableName, table_data = row_data)
                cur.execute(tableDataInsertionQuery)
            
            con.commit()
            
            if fileType == "before":
                comparisonList.append((tableName, tableColumns))

    if fileType == "before":
        
        resultList[dbName] = list()
        resultFileSheetName = dbName[len(FILEPATH + "/" + SHEET_NAME_EXTRACTOR): ]
        if not os.path.isfile(RESULT_FILENAME):
            wb = openpyxl.Workbook()
            wb.save(RESULT_FILENAME)
        wb = load_workbook(filename = RESULT_FILENAME)
        ws = wb.create_sheet(title = resultFileSheetName)
        wb.save(RESULT_FILENAME)
        
        for tableComparison in comparisonList:

            beforeTable = tableComparison[0]
            afterTable = tableComparison[0][:-len("before")] + "after"
            columns = tableComparison[1]
            
            tableResult = dict()
            tableResult["Table Name"] = beforeTable[:-len("_before")]
            tableResult["Columns"] = columns
            
            displayColumns1 = "b." + columns[0] + " AS before_date, a." + columns[0] + " AS after_date, "
            displayColumns2 = "a." + columns[0] + " AS before_date, b." + columns[0] + " AS after_Date, "
            for column in columns[1:-1]:
                displayColumns1 += "a." + column + ", "
                displayColumns2 += "a." + column + ", "
            displayColumns2 += "a." + columns[-1] + " AS count_before, b." + columns[-1] + " AS count_after"
            displayColumns1 += "b." + columns[-1] + " AS count_before, a." + columns[-1] + " AS count_after"
            
            joinCondition = ""
            for column in columns[1:-2]:
                joinCondition += "a." + column + " = b." + column + " AND "
            joinCondition += "a." + columns[-2] + " = b." + columns[-2]
            
            query1 = "SELECT {display_columns} from {after_table} AS a LEFT JOIN {before_table} AS b ON {join_condition}"
            query1 = query1.format(display_columns = displayColumns1, after_table = afterTable, before_table = beforeTable, join_condition = joinCondition)
            query2 = "SELECT {display_columns} from {before_table} AS a LEFT JOIN {after_table} AS b ON {join_condition}"
            query2 = query2.format(display_columns = displayColumns2, before_table = beforeTable, after_table = afterTable, join_condition = joinCondition)
            query = "{} UNION {}".format(query1, query2)
            
            new_df = pd.read_sql(query, con)
            new_df[["count_before","count_after"]] = new_df[["count_before","count_after"]].fillna(0)
            new_df[["count_before","count_after"]] = new_df[["count_before","count_after"]].applymap(int)
            new_df["Difference (after - before)"] = new_df["count_after"] - new_df["count_before"]
            new_df = new_df.sort_values(new_df.columns[2:-3].tolist())
            
            # Write and style the table name
            wb = load_workbook(RESULT_FILENAME)
            ws = wb[resultFileSheetName]
            currentRow = ws.max_row + 2
            tableNameCell = "B" + str(currentRow)
            ws[tableNameCell] = afterTable[:-len("_after")]
            ws.merge_cells(tableNameCell + ":" + get_column_letter(new_df.shape[1] + 1) + str(currentRow))
            mergedCell = ws[tableNameCell]
            mergedCell.font = Font(bold = True, color = "00FFFFFF")
            mergedCell.fill = PatternFill("solid", fgColor = "00003366")
            mergedCell.alignment = Alignment(horizontal = "center", vertical = "center")
            wb.save(RESULT_FILENAME)
            
            # Add the comparison data
            append_df_to_excel(filename = RESULT_FILENAME, df = new_df, sheet_name = resultFileSheetName, startrow = currentRow, index = True)
            
            # Add styling to highlight the difference
            wb = load_workbook(RESULT_FILENAME)
            ws = wb[resultFileSheetName]
            currentRow += 1
            maxRow = ws.max_row
            g.formula = ['ISBLANK($C' + str(currentRow) + ')']
            r.formula = ['ISBLANK($B' + str(currentRow) + ')']
            ws.conditional_formatting.add("B" + str(currentRow) + ":" + get_column_letter(new_df.shape[1] + 1) + str(maxRow), r)
            ws.conditional_formatting.add("B" + str(currentRow) + ":" + get_column_letter(new_df.shape[1] + 1) + str(maxRow), g)
            for index in range(currentRow, maxRow + 1):
                ws[get_column_letter(new_df.shape[1] + 1) + str(index)].number_format = '#,##0'
            ws.column_dimensions['A'].hidden = True
            wb.save(RESULT_FILENAME)
            
    con.close()
    file.close()
    
    if fileType == "before":
        os.remove(dbName + '.db')

# Deleting the default sheet created by openpxl
wb = load_workbook(RESULT_FILENAME)
del wb["Sheet"]
wb.save(RESULT_FILENAME)